In [2]:
import ipynb.fs.full.FootballApi as api
import ipynb.fs.full.PostgreDB as pg
import json

In [3]:
country = 'Brazil'

In [26]:
conn = pg.PostgreDB.connect()

In [5]:
# selecionado id do Pais
sql = f" SELECT name, id FROM country WHERE name = '{country}' "
cur = pg.PostgreDB.run_query(conn, sql)
result = cur.fetchone()

if result: 
    country_name = result[0]
    country_id = result[1]

    print(f"Country found! [{country_name}][{country_id}] ")

else:
    print("COUNTRY NOT FOUND! ERROR")

Country found! [Brazil][185] 


In [6]:
#importando season
teams = api.FootballApi(function=f"teams?country={country}").get_data()

In [27]:
for team in teams['response']:

    team_id = team["team"]['id']
    team_name = team["team"]['name']
    team_code = team["team"]['code']
    team_founded = team["team"]['founded']
    team_national = team["team"]['national']
    team_logo = team["team"]['logo']

    stadium_id = team["venue"]['id']
    stadium_name = team["venue"]['name']
    stadium_address = team["venue"]['address']
    stadium_city = team["venue"]['city']
    stadium_capacity = team["venue"]['capacity']
    stadium_surface = team["venue"]['surface']
    stadium_image = team["venue"]['image']
    
    if stadium_city:

        stadium_city_split = stadium_city.split(sep=",", maxsplit=1)
        
        city = stadium_city_split[0].strip()
        #nem todas as cidade vem com estado, as que nao vem, setamos a cidade tambem no estado
        if len(stadium_city_split) > 1:
            state = stadium_city_split[1].strip()
        else:
            state = stadium_city_split[0].strip()

        # ---------------------------------------------------------------------------
        # inserindo cidades novas
        sql = f" INSERT INTO public.city (name, state, id_country) VALUES('{city}', '{state}', {country_id}) ON CONFLICT DO NOTHING RETURNING id; "
        #print(sql)
        cur1 = pg.PostgreDB.run_query(conn, sql)
        res1 = cur1.fetchone()
        if res1:
            id_city = res1[0]
        else:
            sql = f"SELECT id FROM public.city WHERE name='{city}' AND state='{state}'"
            cur12 = pg.PostgreDB.run_query(conn, sql)
            res12 = cur12.fetchone()
            id_city = res12[0]

        # ---------------------------------------------------------------------------
        # inserindo estadios novos
        sql = f""" INSERT INTO public.stadium (id, name, adress, capacity, surface, image, id_city) 
                    VALUES({stadium_id} ,'{stadium_name.replace("'","''")}', '{stadium_address}', {stadium_capacity}, '{stadium_surface}', '{stadium_image}', {id_city})
                    ON CONFLICT DO NOTHING RETURNING id;""";
        cur2 = pg.PostgreDB.run_query(conn, sql)
        res2 = cur2.fetchone()
        if res2:
            id_stadium = res2[0]
        else:
            id_stadium = stadium_id

    # QUER INSERT TEAM

    #sql = f" INSERT INTO season (season) VALUES ({season}) ON CONFLICT DO NOTHING; "
    # print(sql)
    #pg.PostgreDB.run_query(conn, sql)
    
pg.PostgreDB.commit(conn)
pg.PostgreDB.close(conn)

 INSERT INTO public.stadium (id, name, adress, capacity, surface, image, id_city) 
                    VALUES(204 ,'Estadio Jornalista Mário Filho (Maracanã)', 'Rua Professor Eurico Rabelo, Maracanã', 78838, 'grass', 'https://media.api-sports.io/football/venues/204.png', 433)
                    ON CONFLICT DO NOTHING RETURNING id;
 INSERT INTO public.stadium (id, name, adress, capacity, surface, image, id_city) 
                    VALUES(216 ,'Arena Fonte Nova', 'Rua Lions Club, Nazaré', 56500, 'grass', 'https://media.api-sports.io/football/venues/216.png', 434)
                    ON CONFLICT DO NOTHING RETURNING id;
 INSERT INTO public.stadium (id, name, adress, capacity, surface, image, id_city) 
                    VALUES(244 ,'Estádio José Pinheiro Borda', 'Avenida Padre Cacique 891, Bairro Menino Deus', 50128, 'grass', 'https://media.api-sports.io/football/venues/244.png', 435)
                    ON CONFLICT DO NOTHING RETURNING id;
 INSERT INTO public.stadium (id, name, adres

UndefinedColumn: column "none" does not exist
LINE 2:                     VALUES(None ,'Estádio Municipal Alziro d...
                                   ^
HINT:  Perhaps you meant to reference the column "stadium.name".


In [ ]:
create table team(
ID INTEGER PRIMARY KEY,
NAME VARCHAR(200),
ID_COUNTRY INTEGER,
YEAR_FOUNDED SMALLINT,
national BOOL,
LOGO VARCHAR(250),
ID_STADIUM INTEGER,
   CONSTRAINT fk_country
      FOREIGN KEY(ID_COUNTRY) 
	  REFERENCES country(id)
	  ON DELETE NO action,
   CONSTRAINT fk_stadium
      FOREIGN KEY(ID_STADIUM) 
	  REFERENCES stadium(id)
	  ON DELETE NO ACTION
);
comment on column team.national is 'Brazil, EUA, Spain, Italy...';
comment on column team.LOGO is 'URL';